In [2]:
import pandas as pd

## Reading Data

In [6]:
books_review = pd.read_csv('../data/goodreads.csv')
books_genre = pd.read_csv('../data/goodreads_w_genre.csv')

# Format columns name
books_review = books_review.rename(columns = {"bookID": "id", "authors": "author(s)", "  num_pages": "num_pages"})
books_genre.columns = ['id', 'title', 'author(s)', 'description', 'genres', 'avg_rating', 'num_ratings', 'url']

# Filter columns from books reviews
books_review = books_review[['title', 'author(s)', 'average_rating', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher']]

### Remove duplicates with the keys Title, Author

In [9]:
def string_col_processing(df, col):
    """
    Function to process a string column
    :param df: Dataframe
    :param col: column name to process
    :return: return a pandas series with the processed column
    """
    return df[col].str.lower().str.replace('[^a-zA-Z0-9 ]+', ' ').str.strip()

for df in [books_genre, books_review]:
    for col in ['title', 'author(s)']:
        df[col] = string_col_processing(df, col)

In [10]:
def drop_duplicates_author_title(df):
    """
    Drop duplicates from a dataframe based on the combination of title and authors(s)
    :param df:
    :return: dataframe
    """
    return df.drop_duplicates(subset=['title', 'author(s)'])

books_review = drop_duplicates_author_title(books_review)
books_genre = drop_duplicates_author_title(books_genre)

In [11]:
print(books_review.shape[0])
print(books_genre.shape[0])

10795
9895


### Get a list of valid genres from the book dataset that contains the genre

In [12]:
import json

# The column genres is a string with a list of genres, so we are using json module to convert it to a list
genre_set = set()
for row in books_genre.genres.to_list():
    # Lowercase the string to avoid case sensitivity
    row_lower = row.lower()
    genre_list = json.loads(row_lower.replace("'", '"'))
    genre_set = genre_set | set(genre_list)

print(f"Number of genres: {len(genre_set)}")

Number of genres: 617


### Using Open Library API to get the genre from the books review

In [13]:
# Import request for usage
import requests

def get_genre_open_library(isbn, genre_set):
    """
    Function to get a genre list from an ISBN using Open Library API
    :param isbn: 13 char string that characterizes a book
    :param genre_set: set of valid genres extracted from genre dataset
    :return:
    """
    base_url = "https://openlibrary.org"

    # Get the edition JSON for a given ISBN
    edition_resp = requests.get(f"{base_url}/isbn/{isbn}.json")
    if edition_resp.status_code == 404:
        return []
    edition = edition_resp.json()

    # Get the work key from the edition
    works = edition.get("works", [])
    if not works:
        return []

    # Get the first work key
    work_key = works[0]["key"]

    # Get the work JSON for the work key
    work_resp = requests.get(f"{base_url}{work_key}.json")
    work_resp.raise_for_status()
    work = work_resp.json()

    # Get the subjects from the work
    subjects = work.get("subjects", [])

    genre_aux_list = []
    for subject in subjects:
        if subject.lower() in genre_set:
            genre_aux_list.append(subject.lower())

    return genre_aux_list

### Getting the genre list

In [ ]:
# Import time module to avoid rate limit
import time

genre_data = {}
# For each ISBN, get the genre list
for isbn in books_review['isbn13']:
    if len(str(isbn)) == 13:
        print(isbn)
        # Create a new column in the dataframe for the genre list
        res = get_genre_open_library(isbn, genre_set)
        genre_data[isbn] = res
        # Cooldown time
        time.sleep(2)

In [27]:
genre_df = pd.DataFrame(data = [genre_data.keys(),genre_data.values()]).transpose()
genre_df.columns = ['isbn13','genres']

print(genre_df.shape)

(10786, 11)


In [28]:
books_final_df = pd.merge(books_review, genre_df, on='isbn13')
print(books_final_df.shape)

(10770, 20)


In [19]:
books_final_df.to_csv("../data/final_df.csv", index=False)

In [20]:
books_final_df = pd.read_csv("../data/final_df.csv")

In [21]:
books_final_df.head()

,title,author(s),average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres
0,harry potter and the half-blood prince (harry ...,j.k. rowling/mary grandpré,4.57,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"['fiction', 'magic', 'witches', 'school storie..."
1,harry potter and the order of the phoenix (har...,j.k. rowling/mary grandpré,4.49,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"['magic', 'fiction', 'fantasy', 'coming of age..."
2,harry potter and the chamber of secrets (harry...,j.k. rowling,4.42,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"['school stories', 'fiction', 'fantasy', 'magi..."
3,harry potter and the prisoner of azkaban (harr...,j.k. rowling/mary grandpré,4.56,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"['fantasy', 'literature', 'adventure', 'magic'..."
4,harry potter boxed set books 1-5 (harry potte...,j.k. rowling/mary grandpré,4.78,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,[]
